#  Segmenting and Clustering Neighborhoods in Toronto

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information.. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Import Pandas and numpy

In [2]:
import pandas as pd
import numpy as np

## Module 1

### Get the data from wikipedia page and create dataframe


### Importing BeautifulSoup and requests

In [3]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

Creating a variable "url" to store the wikipedia link

In [4]:
#url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"   ---- Not working , Data has been updated on the wiki
# Using old version of the url with data in the correct required format
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=1012023397'

Using padas read_html function to read the url data

In [5]:
data = pd.read_html(url, flavor='bs4')

Create dataframe df and input first table from the data

In [6]:
df=data[0]

In [7]:
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [8]:
df.shape

(180, 3)

### Removing the rows in which Borough is Not assigned

In [9]:
df = df[df["Borough"] != "Not assigned"]

In [10]:
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
print('Shape: ', df.shape)
print('Check if any assigned value present for Borough: ', df.Neighbourhood.str.count("Not assigned").sum())

Shape:  (103, 3)
Check if any assigned value present for Borough:  0


**If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough**

**No NaN value present in Neighbourhood**

In [12]:
df.Neighbourhood.str.count("Not assigned").sum()

0

**More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table**

In [13]:
df = df.groupby(['Postal Code','Borough']).Neighbourhood.apply(lambda x: ', '.join(x)).to_frame()
df.reset_index(inplace=True)
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Check the shape of our final dataframe

In [14]:
print('Share of Dataframe df: ', df.shape)

Share of Dataframe df:  (103, 3)


## Module 2

**Import Geocoder**

In [15]:
import geocoder # import geocoder

In [16]:
# Read and store the csv from the link provided:

data = pd.read_csv("https://cocl.us/Geospatial_data")
data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Check the shape of postal code dataframe and geospatial dataframe

Both are same in shape

In [17]:
print("The shape of Postal Code Dataframe: ", df.shape)
print("the shape of Geospatial csv Dataframe: ", data.shape)

The shape of Postal Code Dataframe:  (103, 3)
the shape of Geospatial csv Dataframe:  (103, 3)


In [18]:
df.dtypes

Postal Code      object
Borough          object
Neighbourhood    object
dtype: object

**Merge the two dataframes**

In [19]:
merged_df = df.join(data.set_index('Postal Code'), on='Postal Code', how='inner')
merged_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


**Shape of the merged dataset**

In [20]:
print('The shape of the merged dataset: ', merged_df.shape)

The shape of the merged dataset:  (103, 5)


## Module 3

## Importing geopy and folium

In [21]:
# Uncomment this if Geopy is not installed
#!pip install geopy
from geopy.geocoders import Nominatim

In [22]:
# Uncomment this if Folium is not installed
#!pip install folium
import folium

Find the coordinates of the Toronto:

Use the nominatim , pass the required paramters and get the coordinates

In [23]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent='toronto_exploration')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto :  {}, {}.'
      .format(latitude, longitude))

The coordinates of Toronto :  43.6534817, -79.3839347.


Creating the map centered in Toronto with the neighborhood marked:

In [24]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add boroughs as markers
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Creating the details required for Foursquare:

In [25]:
CLIENT_ID = 'XKIJ30QKHDEHRRH0IS0TOZJLEJZ10FY0Z0ZMTR00HBT2XKJC' 
CLIENT_SECRET = 'OJ033FI1K5ICVQAHPEJBLC0USERCDVYUCW5AQDDSWZUIIPVU'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XKIJ30QKHDEHRRH0IS0TOZJLEJZ10FY0Z0ZMTR00HBT2XKJC
CLIENT_SECRET:OJ033FI1K5ICVQAHPEJBLC0USERCDVYUCW5AQDDSWZUIIPVU


Function to extract category of the venue

In [26]:
# Function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Function to get nearby venues

In [27]:
# Function that gets 100 closest venues to a postal code coordinates
def getNearbyVenues(postal_codes, latitudes, longitudes, radius=800):
    
    LIMIT = 100
    venues_list=[]
    
    for code, lat, lng in zip(postal_codes, latitudes, longitudes):
        print('Querying venues from: ', code)
            
        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # Return only relevant information for each nearby venue
        venues_list.append([(
            code, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Neighborhood Group Latitude', 
                  'Neighborhood Group Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
filtered_df = merged_df[merged_df.Borough.str.contains('Toronto',case=False)]
filtered_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [29]:
print('Shape : ', filtered_df.shape)

Shape :  (40, 5)


In [30]:
all_venues = getNearbyVenues(filtered_df['Postal Code'],
                             filtered_df['Latitude'],
                             filtered_df['Longitude']
                            )


print('Data shape: ', all_venues.shape)
all_venues.head()

Querying venues from:  M4E
Querying venues from:  M4K
Querying venues from:  M4L
Querying venues from:  M4M
Querying venues from:  M4N
Querying venues from:  M4P
Querying venues from:  M4R
Querying venues from:  M4S
Querying venues from:  M4T
Querying venues from:  M4V
Querying venues from:  M4W
Querying venues from:  M4X
Querying venues from:  M4Y
Querying venues from:  M5A
Querying venues from:  M5B
Querying venues from:  M5C
Querying venues from:  M5E
Querying venues from:  M5G
Querying venues from:  M5H
Querying venues from:  M5J
Querying venues from:  M5K
Querying venues from:  M5L
Querying venues from:  M5N
Querying venues from:  M5P
Querying venues from:  M5R
Querying venues from:  M5S
Querying venues from:  M5T
Querying venues from:  M5V
Querying venues from:  M5W
Querying venues from:  M5X
Querying venues from:  M6G
Querying venues from:  M6H
Querying venues from:  M6J
Querying venues from:  M6K
Querying venues from:  M6N
Querying venues from:  M6P
Querying venues from:  M6R
Q

,Postal Code,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,M4E,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
3,M4E,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
4,M4E,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater


In [31]:
all_venues.groupby('Postal Code').count()

,Neighborhood Group Latitude,Neighborhood Group Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,
M4E,47,47,47,47,47,47
M4K,100,100,100,100,100,100
M4L,51,51,51,51,51,51
M4M,93,93,93,93,93,93
M4N,7,7,7,7,7,7
M4P,56,56,56,56,56,56
M4R,37,37,37,37,37,37
M4S,78,78,78,78,78,78
M4T,13,13,13,13,13,13


**Unique Categories**

In [32]:
print('Unique categories.',format(len(all_venues['Venue Category'].unique())))

Unique categories. 285


Applying One-Hot Encoding for venuw category

In [33]:
# Applying OH Encoding by 'creating dummies'
venues_one_hot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# Add the 'Postal Code' column back to dataset
venues_one_hot['Postal Code'] = all_venues['Postal Code'] 

# Move postal code column to the first column
fixed_columns = [venues_one_hot.columns[-1]] + list(venues_one_hot.columns[:-1])
venues_one_hot = venues_one_hot[fixed_columns]

venues_one_hot.head()

,Postal Code,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group by Postal code and by taking the mean of the frequency of occurrence of each category:

In [34]:
venues_grouped = venues_one_hot.groupby('Postal Code').mean().reset_index()
venues_grouped

,Postal Code,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,M4E,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.021277,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,M4K,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000
2,M4L,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.019608,0.000000
3,M4M,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,...,0.00,0.000000,0.00000,0.000000,0.021505,0.00,0.000000,0.000000,0.000000,0.021505
4,M4N,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,M4P,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.017857,0.00000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000
6,M4R,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.027027
7,M4S,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.012821,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.012821
8,M4T,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
9,M4V,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016393,...,0.00,0.016393,0.00000,0.000000,0.016393,0.00,0.000000,0.000000,0.000000,0.016393


The 5 more popular venue categories from each neighborhood group:

In [35]:
num_top_venues = 5

for code in venues_grouped['Postal Code']:
    print("------------ "+code+" ------------")
    temp = venues_grouped[venues_grouped['Postal Code'] == code].T.reset_index()
    temp.columns = ['Venue Cat','Freq']
    temp = temp.iloc[1:]
    temp['Freq'] = temp['Freq'].astype(float)
    temp = temp.round({'Freq': 2})
    print(temp.sort_values('Freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

------------ M4E ------------
             Venue Cat  Freq
0                  Pub  0.11
1       Breakfast Spot  0.04
2                  Bar  0.04
3  Japanese Restaurant  0.04
4       Sandwich Place  0.04


------------ M4K ------------
            Venue Cat  Freq
0    Greek Restaurant  0.11
1         Coffee Shop  0.08
2                 Pub  0.05
3                Café  0.04
4  Italian Restaurant  0.03


------------ M4L ------------
              Venue Cat  Freq
0     Indian Restaurant  0.12
1  Fast Food Restaurant  0.06
2            Restaurant  0.04
3                   Gym  0.04
4        Sandwich Place  0.04


------------ M4M ------------
     Venue Cat  Freq
0         Café  0.05
1  Coffee Shop  0.05
2       Bakery  0.05
3          Bar  0.04
4        Diner  0.04


------------ M4N ------------
              Venue Cat  Freq
0             Bookstore  0.14
1           Coffee Shop  0.14
2                  Park  0.14
3                  Café  0.14
4  Gym / Fitness Center  0.14


------------

Function to sort the venues: 

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

We create the new DataFrame, with the first 10 most common venues, each one as a column of the DF:

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create the columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create the new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Postal Code'] = venues_grouped['Postal Code']

for ind in np.arange(venues_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Pub,Grocery Store,Japanese Restaurant,Sandwich Place,Breakfast Spot,Bar,Bagel Shop,Café,Bank,Indie Movie Theater
1,M4K,Greek Restaurant,Coffee Shop,Pub,Café,Spa,Italian Restaurant,Fast Food Restaurant,Yoga Studio,Bookstore,Breakfast Spot
2,M4L,Indian Restaurant,Fast Food Restaurant,Gym,Grocery Store,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Bus Stop,Brewery
3,M4M,Bakery,Café,Coffee Shop,Bar,Brewery,Diner,American Restaurant,Sushi Restaurant,Pizza Place,Restaurant
4,M4N,Restaurant,Bus Line,Bookstore,Café,Gym / Fitness Center,Coffee Shop,Park,Dog Run,Doner Restaurant,Donut Shop


Import required libraries

In [53]:
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Library to evaluate K value in K-Means
from yellowbrick.cluster import KElbowVisualizer

Next, we have to drop the Postal Code column to perform the K-Means algorithm:

In [54]:
venues_clustering = venues_grouped.drop('Postal Code', 1)
venues_clustering.head()

,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.021277,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.020000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.019608,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.032258,0.0,...,0.0,0.000000,0.0,0.0,0.021505,0.0,0.0,0.0,0.000000,0.021505
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000


### Clustering

Run k-means to cluster the neighborhood into 5 clusters

In [55]:
# Best K parameter
k_clusters = 5

# Run K-Means clustering algorithm
model = KMeans(n_clusters=k_clusters, random_state=0).fit(venues_clustering)

# Check cluster labels generated for each row in the dataframe
model.labels_[0:10] 

array([1, 1, 1, 1, 4, 1, 1, 1, 2, 1])

In [56]:
# Add clustering labels
venues_sorted.insert(0, 'Cluster Labels', model.labels_)

merged_venues = filtered_df

# Merge venues_grouped with the DF with the Postal Codes selected
# to add latitude/longitude for each neighborhood group
merged_venues = merged_venues.join(venues_sorted.set_index('Postal Code'), on='Postal Code')

merged_venues.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Grocery Store,Japanese Restaurant,Sandwich Place,Breakfast Spot,Bar,Bagel Shop,Café,Bank,Indie Movie Theater
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Pub,Café,Spa,Italian Restaurant,Fast Food Restaurant,Yoga Studio,Bookstore,Breakfast Spot
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Indian Restaurant,Fast Food Restaurant,Gym,Grocery Store,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Bus Stop,Brewery
43,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Bakery,Café,Coffee Shop,Bar,Brewery,Diner,American Restaurant,Sushi Restaurant,Pizza Place,Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Restaurant,Bus Line,Bookstore,Café,Gym / Fitness Center,Coffee Shop,Park,Dog Run,Doner Restaurant,Donut Shop


In [57]:
merged_venues.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Grocery Store,Japanese Restaurant,Sandwich Place,Breakfast Spot,Bar,Bagel Shop,Café,Bank,Indie Movie Theater
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Pub,Café,Spa,Italian Restaurant,Fast Food Restaurant,Yoga Studio,Bookstore,Breakfast Spot
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Indian Restaurant,Fast Food Restaurant,Gym,Grocery Store,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Bus Stop,Brewery
43,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Bakery,Café,Coffee Shop,Bar,Brewery,Diner,American Restaurant,Sushi Restaurant,Pizza Place,Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Restaurant,Bus Line,Bookstore,Café,Gym / Fitness Center,Coffee Shop,Park,Dog Run,Doner Restaurant,Donut Shop


Visualize the resulting clusters:

In [58]:
# Create a Folium map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Set the color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_venues['Latitude'], merged_venues['Longitude'], merged_venues['Postal Code'], merged_venues['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1:

In [59]:
merged_venues.loc[merged_venues['Cluster Labels'] == 0, merged_venues.columns[[1,2] + list(range(5, merged_venues.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,Roselawn,0,Home Service,Playground,Garden,Pet Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Farm


#### Cluster 2:

In [60]:
merged_venues.loc[merged_venues['Cluster Labels'] == 1, merged_venues.columns[[1,2] + list(range(5, merged_venues.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,The Beaches,1,Pub,Grocery Store,Japanese Restaurant,Sandwich Place,Breakfast Spot,Bar,Bagel Shop,Café,Bank,Indie Movie Theater
41,East Toronto,"The Danforth West, Riverdale",1,Greek Restaurant,Coffee Shop,Pub,Café,Spa,Italian Restaurant,Fast Food Restaurant,Yoga Studio,Bookstore,Breakfast Spot
42,East Toronto,"India Bazaar, The Beaches West",1,Indian Restaurant,Fast Food Restaurant,Gym,Grocery Store,Coffee Shop,Restaurant,Sandwich Place,Italian Restaurant,Bus Stop,Brewery
43,East Toronto,Studio District,1,Bakery,Café,Coffee Shop,Bar,Brewery,Diner,American Restaurant,Sushi Restaurant,Pizza Place,Restaurant
45,Central Toronto,Davisville North,1,Coffee Shop,Italian Restaurant,Pizza Place,Thai Restaurant,Café,Gym,Deli / Bodega,Ramen Restaurant,Park,Sushi Restaurant
46,Central Toronto,"North Toronto West, Lawrence Park",1,Coffee Shop,Thai Restaurant,Skating Rink,Café,Italian Restaurant,Restaurant,Diner,Sporting Goods Shop,Playground,Salon / Barbershop
47,Central Toronto,Davisville,1,Italian Restaurant,Coffee Shop,Sushi Restaurant,Café,Restaurant,Sandwich Place,Gym,Pizza Place,Dessert Shop,Thai Restaurant
49,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",1,Coffee Shop,Sushi Restaurant,Thai Restaurant,Italian Restaurant,Grocery Store,Sandwich Place,Spa,Bank,Bagel Shop,Café
51,Downtown Toronto,"St. James Town, Cabbagetown",1,Coffee Shop,Restaurant,Café,Grocery Store,Pizza Place,Thai Restaurant,Park,Beer Store,Diner,Bakery
52,Downtown Toronto,Church and Wellesley,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Café,Italian Restaurant,Diner,Park,Yoga Studio,Restaurant,Thai Restaurant


#### Cluster 3:

In [61]:
merged_venues.loc[merged_venues['Cluster Labels'] == 2, merged_venues.columns[[1,2] + list(range(5, merged_venues.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,"Moore Park, Summerhill East",2,Park,Grocery Store,Thai Restaurant,Sandwich Place,Candy Store,Gym / Fitness Center,Gym,Japanese Restaurant,Café,Tennis Court


#### Cluster 4:

In [62]:
merged_venues.loc[merged_venues['Cluster Labels'] == 3, merged_venues.columns[[1,2] + list(range(5, merged_venues.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,Rosedale,3,Park,Trail,Playground,Bank,Grocery Store,Candy Store,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant


#### Cluster 5:

In [63]:
merged_venues.loc[merged_venues['Cluster Labels'] == 4, merged_venues.columns[[1,2] + list(range(5, merged_venues.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,Lawrence Park,4,Restaurant,Bus Line,Bookstore,Café,Gym / Fitness Center,Coffee Shop,Park,Dog Run,Doner Restaurant,Donut Shop
